In [1]:
#importing the requests library
import requests

Using [marketstack](https://marketstack.com/?utm_source=Geekflare&utm_medium=LeadsAcquisition&utm_content=Listing) to make API calls to retrieve end-of-day stock price of your preferred company.

API documentations can be found [here](https://marketstack.com/documentation). Code examples for different languages can be found [here](https://marketstack.com/documentation#php)

In [78]:
companies = ['GOOGL','APPL','FB','AMZN']

for symbol in companies:
    url = 'http://api.marketstack.com/v1/eod'
    params = {
        'access_key' : '038908b69357069eadcfc963cbafe6ee',
        'symbols' : symbol,
        'limit':1000 
            }
    resp = requests.get(url,params=params)
    data = resp.json()

In [80]:
#getting the list of end of day price
price_list = []
for i in data['data']:
        price_list.append(i['close'])

In [85]:
companies = ['GOOGL','APPL','FB','AMZN']
print('_price_list'.join(companies))

GOOGL_price_listAPPL_price_listFB_price_listAMZN


In [86]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.
